### This is a test of using a black-box likelihood function in pymc3 based on this guide : https://docs.pymc.io/notebooks/blackbox_external_likelihood.html

In [1]:
%matplotlib inline
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
print('Running on PyMC3 v{}'.format(pm.__version__))

# for reproducibility here's some version info for modules used in this notebook
import theano
import theano.tensor as tt
import platform
import cython
import IPython
import matplotlib
import emcee
import corner
import os
print("Python version:     {}".format(platform.python_version()))
print("IPython version:    {}".format(IPython.__version__))
print("Numpy version:      {}".format(np.__version__))
print("Theano version:     {}".format(theano.__version__))
print("Matplotlib version: {}".format(matplotlib.__version__))
print("emcee version:      {}".format(emcee.__version__))
print("corner version:     {}".format(corner.__version__))

Running on PyMC3 v3.8
Python version:     3.7.6
IPython version:    7.13.0
Numpy version:      1.18.1
Theano version:     1.0.4
Matplotlib version: 3.2.1
emcee version:      2.2.1
corner version:     2.0.1


In [40]:
from configurations import *

SystemsInfo = {"{:s}-{:s}-{:d}".format(*s): systems_setting(*s) for s in systems}

if 'Pb-Pb-2760' in system_strs:
    SystemsInfo["Pb-Pb-2760"]["run_id"] = "production_500pts_Pb_Pb_2760"
    SystemsInfo["Pb-Pb-2760"]["n_design"] = 500
    SystemsInfo["Pb-Pb-2760"]["n_validation"] = 100
    SystemsInfo["Pb-Pb-2760"]["design_remove_idx"]=list(delete_design_pts_set)
    SystemsInfo["Pb-Pb-2760"]["npc"]=10
    SystemsInfo["Pb-Pb-2760"]["MAP_obs_file"]=str(workdir/'model_calculations/MAP') + '/' + idf_label_short[idf] + '/Obs/obs_Pb-Pb-2760.dat'


if 'Au-Au-200' in system_strs:
    SystemsInfo["Au-Au-200"]["run_id"] = "production_500pts_Au_Au_200"
    SystemsInfo["Au-Au-200"]["n_design"] = 500
    SystemsInfo["Au-Au-200"]["n_validation"] = 100
    SystemsInfo["Au-Au-200"]["design_remove_idx"]=list(delete_design_pts_set)
    SystemsInfo["Au-Au-200"]["npc"] = 6
    SystemsInfo["Au-Au-200"]["MAP_obs_file"]=str(workdir/'model_calculations/MAP') + '/' + idf_label_short[idf] + '/Obs/obs_Au-Au-200.dat'

if 'Pb-Pb-5020' in system_strs:
    SystemsInfo["Pb-Pb-5020"]["MAP_obs_file"]=str(workdir/'model_calculations/MAP') + '/' + idf_label_short[idf] + '/Obs/obs_Pb-Pb-5020.dat'

s = 'Pb-Pb-2760'
#print("SystemsInfo = ")
#print(SystemsInfo)

#design, design_max, design_min, labels = prepare_emu_design(s)
design, design_max, design_min, labels = load_design(system_str=s, pset='main')

Loading main points from production_designs/500pts/design_pts_Pb_Pb_2760_production/design_points_main_PbPb-2760.dat
Loading main ranges from production_designs/500pts/design_pts_Pb_Pb_2760_production/design_ranges_main_PbPb-2760.dat


KeyError: 'labels'

In [2]:
#import our emulator, likelihood, etc...
import emulator
from bayes_mcmc import Chain, Y_exp_data

/Users/derek/miniconda3/envs/8805-old-emcee-env/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/Users/derek/miniconda3/envs/8805-old-emcee-env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/derek/miniconda3/envs/8805-old-emcee-env/lib/python3.7/sit

Using idf = 0 : Grad
can't load design point labels
SystemsInfo = 
{'Pb-Pb-2760': {'proj': 'Pb', 'targ': 'Pb', 'sqrts': 2760, 'main_design_file': 'production_designs/500pts/design_pts_Pb_Pb_2760_production/design_points_main_PbPb-2760.dat', 'main_range_file': 'production_designs/500pts/design_pts_Pb_Pb_2760_production/design_ranges_main_PbPb-2760.dat', 'validation_design_file': 'production_designs/500pts/design_pts_Pb_Pb_2760_production/design_points_validation_PbPb-2760.dat', 'validation_range_file': 'production_designs/500pts/design_pts_Pb_Pb_2760_production//design_ranges_validation_PbPb-2760.dat', 'main_events_dir': 'model_calculations/production_500pts_Pb_Pb_2760/Events/main', 'validation_events_dir': 'model_calculations/production_500pts_Pb_Pb_2760/Events/validation', 'main_obs_file': 'model_calculations/production_500pts_Pb_Pb_2760/Obs/main.dat', 'validation_obs_file': 'model_calculations/production_500pts_Pb_Pb_2760/Obs/validation.dat', 'n_design': 500, 'n_validation': 100, 'de

In [13]:
# define a theano Op for our likelihood function
class LogLike(tt.Op):

    """
    Specify what type of object will be passed and returned to the Op when it is
    called. In our case we will be passing it a vector of values (the parameters
    that define our model) and returning a single "scalar" value (the
    log-likelihood)
    """
    itypes = [tt.dvector] # expects a vector of parameter values when called
    otypes = [tt.dscalar] # outputs a single scalar value (the log likelihood)

    def __init__(self, loglike, data):
        """
        Initialise the Op with various things that our log-likelihood function
        requires. Below are the things that are needed in this particular
        example.

        Parameters
        ----------
        loglike:
            The log-likelihood (or whatever) function we've defined
        data:
            The "observed" data that our log-likelihood function takes in
        x:
            The dependent variable (aka 'x') that our model requires
        sigma:
            The noise standard deviation that our function requires.
        """

        # add inputs as class attributes
        self.likelihood = loglike
        self.data = data
        
        #print("self.data = ")
        #print(self.data)
        
        print("__init__ finished")

    def perform(self, node, inputs, outputs):
        print("perform starting")
        # the method that is used when calling the Op
        theta, = inputs  # this will contain my variables
        print("theta set")

        # call the log-likelihood function
        logl = self.likelihood(theta, self.data)
        print("called logl")

        outputs[0][0] = np.array(logl) # output the log-likelihood
        
        print("perform finished")

In [14]:
ndraws = 3000  # number of draws from the distribution
nburn = 1000   # number of "burn-in points" (which we'll discard)

# create our Op
logl = LogLike(Chain.log_likelihood, Y_exp_data)

# use PyMC3 to sample from log-likelihood
with pm.Model():
    # uniform priors on m and c
    #m = pm.Uniform('m', lower=-10., upper=10.)
    #c = pm.Uniform('c', lower=-10., upper=10.)
    
    # convert m and c to a tensor vector
    #theta = tt.as_tensor_variable([m, c])
    theta = [ pm.Uniform(lower=-10., upper=10.) for ]
    theta = tt.as_tensor_variable()

    # use a DensityDist (use a lamdba function to "call" the Op)
    pm.DensityDist('likelihood', lambda v: logl(v), observed={'v': theta})

    trace = pm.sample(ndraws, tune=nburn, discard_tuned_samples=True)

# plot the traces
_ = pm.traceplot(trace)

# put the chains in an array (for later!)
#samples_pymc3 = np.vstack((trace['m'], trace['c'])).T

__init__ finished
perform starting
theta set


TypeError: '>' not supported between instances of 'numpy.ndarray' and 'builtin_function_or_method'